In [ ]:
import zipfile
file=zipfile.ZipFile('trainingandtestdata.zip')
file.extractall()

In [1]:
import pandas as pd
import numpy as np

columns=['Polarity', 'ID', 'Date', 'Query', 'User', 'Text']

df_train = pd.read_csv('training.1600000.processed.noemoticon.csv', encoding='iso-8859-1', header=None, names=columns)
df_test = pd.read_csv('testdata.manual.2009.06.14.csv', encoding='iso-8859-1', header=None, names=columns)

# df_train.columns=columns

In [2]:
df_train.head()
#df_test.head()

,Polarity,ID,Date,Query,User,Text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [3]:
np.unique(df_test['Query'])
df_test[df_test.Query == 'NO_QUERY']

,Polarity,ID,Date,Query,User,Text


In [4]:
df_train.isnull().sum()

Polarity    0
ID          0
Date        0
Query       0
User        0
Text        0
dtype: int64

In [5]:
np.unique(df_train['Polarity'])

array([0, 4], dtype=int64)

In [6]:
df_train.shape

(1600000, 6)

In [7]:
df_train[df_train.Query != 'NO_QUERY']

,Polarity,ID,Date,Query,User,Text


In [8]:
#df_train['Date'].dt.strftime('%Y-%m-%d %H:%M:%S')
#'Mon Apr 06 22:19:45 PDT 2009'
# '%a %b %d %H:%M:%S %z %y'
df_train['DateTime']=pd.to_datetime(df_train['Date'], format='%a %b %d %H:%M:%S PDT %Y')

In [9]:
X_train=df_train['Text']
y_train=df_train['Polarity']
#X_test=df_test['Text']
#y_test=df_test['Polarity']

In [10]:
import numpy as np
import re
from nltk.corpus import stopwords
stop = stopwords.words('english')
def tokenizer(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text.lower())
    text = re.sub('[\W]+', ' ', text.lower()) + ' '.join(emoticons).replace('-', '')
    tokenized = [w for w in text.split() if w not in stop]
    return tokenized

<>:7: SyntaxWarning: invalid escape sequence '\)'
<>:8: SyntaxWarning: invalid escape sequence '\W'
<>:7: SyntaxWarning: invalid escape sequence '\)'
<>:8: SyntaxWarning: invalid escape sequence '\W'
C:\Users\Shabnam\AppData\Local\Temp\ipykernel_16256\2915367723.py:7: SyntaxWarning: invalid escape sequence '\)'
  emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text.lower())
C:\Users\Shabnam\AppData\Local\Temp\ipykernel_16256\2915367723.py:8: SyntaxWarning: invalid escape sequence '\W'
  text = re.sub('[\W]+', ' ', text.lower()) + ' '.join(emoticons).replace('-', '')


In [77]:
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.linear_model import SGDClassifier
vect = HashingVectorizer(decode_error='ignore', n_features=2**21, preprocessor=None, tokenizer=tokenizer)
# {'perceptron', 'log_loss', 'huber', 'epsilon_insensitive', 'squared_epsilon_insensitive', 'squared_error', 'hinge', 'squared_hinge', 'modified_huber'}
clf = SGDClassifier(loss='log_loss', random_state=1)
#doc_stream = stream_docs(path='movie_data.csv')

In [75]:
import pyprind
pbar = pyprind.ProgBar(1600)
classes = np.array([0, 4])
for i in range(1600):
    #print(1000*i,1000*(i+1))
    X_train_i, y_train_i = X_train.iloc[1000*i:1000*(i+1)],y_train.iloc[1000*i:1000*(i+1)] #get_minibatch(doc_stream, size=1000)
    X_train_i = vect.transform(X_train_i)
    clf.partial_fit(X_train_i, y_train_i.values, classes=classes)
    pbar.update()

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:02:32


In [76]:
new_df_test=df_test[df_test['Polarity']!=2]
X_test=new_df_test['Text']
y_test=new_df_test['Polarity']

X_test = vect.transform(X_test)
print('Accuracy: %.5f' % clf.score(X_test, y_test.values))

Accuracy: 0.50696


- squared_hinge Accuracy: 0.50696
- modified_huber Accuracy: 0.50696
- perceptron Accuracy: 0.50696
- log_loss 0.50696
- huber Accuracy: 0.50696
- epsilon_insensitive Accuracy: 0.50696
- squared_epsilon_insensitive Accuracy: 0.49582
- squared_error Accuracy: 0.47075
- hinge Accuracy: 0.50696

In [21]:
print(np.unique(df_test['Polarity']))
print(df_test.shape)
print(df_test[df_test['Polarity']==2].shape)
print(np.unique(y_train))

[0 2 4]
(498, 6)
(139, 6)
[0 4]


In [22]:
print(df_test[df_test['Polarity']==2].shape)
print(df_test[df_test['Polarity']!=2].shape)
print(df_test.shape)
print(df_test[df_test['Polarity']==4].shape)



Polarity    139
ID          139
Date        139
Query       139
User        139
Text        139
dtype: int64
Polarity    359
ID          359
Date        359
Query       359
User        359
Text        359
dtype: int64
Polarity    498
ID          498
Date        498
Query       498
User        498
Text        498
dtype: int64
Polarity    182
ID          182
Date        182
Query       182
User        182
Text        182
dtype: int64


In [80]:
X_train.apply(tokenizer)

0          [switchfoot, http, twitpic, com, 2y1zl, awww, ...
1          [upset, update, facebook, texting, might, cry,...
2          [kenichan, dived, many, times, ball, managed, ...
3                    [whole, body, feels, itchy, like, fire]
4                      [nationwideclass, behaving, mad, see]
                                 ...                        
1599995                  [woke, school, best, feeling, ever]
1599996    [thewdb, com, cool, hear, old, walt, interview...
1599997                [ready, mojo, makeover, ask, details]
1599998    [happy, 38th, birthday, boo, alll, time, tupac...
1599999    [happy, charitytuesday, thenspcc, sparkscharit...
Name: Text, Length: 1600000, dtype: object

In [82]:
print(y_train[y_train==0].shape)


0         0
1         0
2         0
3         0
4         0
         ..
799995    0
799996    0
799997    0
799998    0
799999    0
Name: Polarity, Length: 800000, dtype: int64